In [1]:
import os
import numpy as np
import open3d as o3d

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
#get all file names from either the Archive folder to sample points from the stl files
#OR get all the file names from the unscaled folder to later scale and store them in scaled folder

def get_fileNames(folder_path):
    
    # create an empty list to store the file names
    file_names = []

    # go through each file in the folder
    for filename in os.listdir(folder_path):
        # add the file name to the list
        file_names.append(filename)

    # print the list of file names
    return(file_names)

In [3]:
#since open3d cannot work with stl files, we have to sample points to either .ply or .pcd extension

def stl_to_ply(fileName_stl, noOfPoints,source_folder,destination_folder):
    mesh = o3d.io.read_triangle_mesh(source_folder+'/'+fileName_stl)
    pcd = mesh.sample_points_poisson_disk(noOfPoints)
    o3d.io.write_point_cloud(destination_folder+'/'+fileName_stl.split('.')[0]+'.ply', pcd)
    return(pcd)

In [4]:
#scales a single point cloud's volume (in ply or pcd format) to the reference dataset

def scaler(fileName, reference, source_folder ,destination_folder):     
    # Load the point clouds
    pcd1 = o3d.io.read_point_cloud(source_folder+'/'+fileName)
    pcd2 = o3d.io.read_point_cloud(reference)

    # Get the bounding box diagonal lengths of each point cloud
    bbox1 = pcd1.get_axis_aligned_bounding_box()
    bbox2 = pcd2.get_axis_aligned_bounding_box()
    diag1 = np.linalg.norm(np.asarray(bbox1.get_max_bound()) - np.asarray(bbox1.get_min_bound()))
    diag2 = np.linalg.norm(np.asarray(bbox2.get_max_bound()) - np.asarray(bbox2.get_min_bound()))

    # Scale the first point cloud to match the diagonal length of the second point cloud
    scale_factor = diag2 / diag1
    pcd1.scale(scale_factor, center=bbox1.get_center())
    
    # Save the scaled point cloud to a file
    o3d.io.write_point_cloud(destination_folder+"/"+fileName, pcd1)

In [5]:
def sampleAllFiles():
    fileNames = get_fileNames("Archive")
    referenceFile = "reference.ply"
    folderNameSource = "Archive"
    folderNameDestination = "unscaled"
    for fileName in fileNames:
        stl_to_pcd(fileName,20_000, folderNameSource, folderNameDestination)

In [6]:
#this function scales all the files in the unscaled folder and stores them in the 
def scaleAllFiles():
    folderNameSource = "unscaled"
    fileNames = get_fileNames(folderNameSource)
    referenceFile = "reference.ply"
    folderNameSource = "unscaled"
    folderNameDestination = "scaled"
    for fileName in fileNames:
        scaler(fileName,referenceFile,folderNameSource, folderNameDestination)

In [63]:
# sampleAllFiles()

In [7]:
# scaleAllFiles()